Testing data positive feedback dynamic (20%)

In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [4]:
user_test_amount = 150
yt_test_amount = 18

In [5]:
def generate_train_test(user_following,user_category,item_category,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('user_category shape ',user_category.shape)
    print('YouTuber_category shape ',item_category.shape)
    user_category_norm = np.zeros(user_category.shape)
    for i in range(len(user_category)):
        user_category_norm[i] = user_category[i]/np.max(user_category[i])
    print('user_category after normalized by max...')
    print('user_category_norm shape ',user_category_norm.shape)
    print('user cateogory norm',user_category_norm)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = item_category[0]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(5)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    
    train_t = []
    train_f = []
    test_t = []
    test_f = []
    
    print('test_idx',test_idx)
    test_pos = -1
    for i in range(num_of_users):
        t_for_train = []
        f_for_train = []
        if i not in test_idx: #if not in test id, just append it to true or false list
            for j in range(88):
                if user_following[i][j] == 1:
                    t_for_train.append(j)
                else:
                    f_for_train.append(j)
            train_t.append(t_for_train)
            train_f.append(f_for_train)

        else: #if in test id, choose 2 true and other 
            test_pos += 1
            temp_t = []
            temp_f = []
            for j in range(88):
                if user_following[i][j] == 1:
                    temp_t.append(j)
                else:
                    temp_f.append(j)
            t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
            f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))

            test_t.append(t_for_test)
            test_f.append(f_for_test)

            #other for training
            t_for_train = [item for item in temp_t if not item in t_for_test]
            f_for_train = [item for item in temp_f if not item in f_for_test]
            train_t.append(t_for_train)
            train_f.append(f_for_train)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f,user_category_norm

In [6]:
train_t,train_f,test_t,test_f,user_category_norm = generate_train_test(user_following,user_category,YouTuber_category,active_users,user_test_amount,yt_test_amount)

Generate_train_test----
user_following shape  (1489, 88)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)
user_category after normalized by max...
user_category_norm shape  (1489, 17)
user cateogory norm [[0.         1.         0.         ... 0.05714286 0.         0.        ]
 [0.24390244 0.         0.02439024 ... 0.         0.         0.09756098]
 [0.04210526 0.04210526 0.05263158 ... 0.02105263 0.         0.02105263]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01408451 0.01408451 0.04225352 ... 0.02816901 0.         0.02816901]
 [0.03703704 0.22222222 0.14814815 ... 0.         0.         0.        ]]
Min number of followings  5
Max number of followings  34
test_idx [3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512,

In [7]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train/len(user_following)
print('training',avg)
#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print('testing',avg)

training 9.758226997985226
testing 5.466666666666667


In [8]:
all_auxilary = [i for i in range(88)]

# Recommendation  Module

In [25]:
n = len(user_following)
m = 88  
k = 64
l = 2939
#embedding_dims = 150

In [17]:
def training(save_name): 
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()
    
    train_yes_id=[] 
    for q in range(7):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yesr=[]
        
            """
            only choose positive 
            """
            #sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            """
            choose all YouTuber 
            """
            sample= all_auxilary #選全部的Positive
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            #user degree of category favor 
            r_3=np.zeros(len(sample)) 
         
            for b in range(len(sample)):
                yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
        
            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            #train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                pos = sample.index(ta)
                
               
                train_f_sample = random.sample(train_f[z],10)
                
                for b in train_f_sample:
                    
                    _a_list,r3,_auc, _loss,_=sess.run([a_list_smooth,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], l_id:sample, l_id_len:[len(sample)],positive_id:train_t[z],positive_len:[len(train_t[z])],r:r_3,
                                        })
                    #print(_a_list)
                    #print(r3)
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
                    
        #print('a_list:',_a_list)
        #print('a_list_soft:',r3)
        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
    print('Total cost ',time.time()-t0,' sec')   
    U, Y, A,Au, Ay, Aa =sess.run([user_latent, item_latent, aux_item,Wu, Wy, Wa])
    #np.savez('../Data/grid_search_weight/Our/'+save_name+'.npz', 
    #                    U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)
    np.savez('../Data/npy/mask_feature/result300/'+save_name+'.npz', 
                        U=U, Y=Y, A=A, Wu=Au, Wy=Ay, Wa=Aa)
    
    return U, Y, A, Au, Ay, Aa

# No feature (Training Part)

In [ ]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]
#only_features = ['Img0.5_ALL']
try_count = [str(try_i) for try_i in range(10)]

testcount = 0
finish_list = []
for tryi in try_count:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished',finish_list)
                    finish_list.append(only_feature)
                    print('Now Dims:',embedding_dims)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))



                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])


                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True)
                                )[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    aux_np = tf.expand_dims(tf.zeros(k),0)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Aur, Ayr, Aar = training('No_feature'+tryi)
                    print('Finish dims:,',embedding_dims)

Finished ['Img0.5_ALL']
Now Dims: 200
Iteraction: 0


# Testing 

In [9]:
import os 
import numpy as np
import random
import math
from sklearn.metrics import average_precision_score
import pandas as pd

In [10]:
def square_sum(pars):
    all_squ_sum = []
    for par in pars:
        all_squ_sum.append(np.sum(np.multiply(par,par)))
    return all_squ_sum 
def shape_sum(pars):
    all_shape = []
    for par in pars:
        all_shape.append(par.shape)
    return all_shape
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))  

In [11]:
#over5 = 0
#for num in YouTuber_followers:
#    if num >= 5:
#        over5+=1
#print('The num of followers over 5:',over5)
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)
print('user cateogory norm',user_category_norm)
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)
test_amount = 150
yt_test_amount = 18
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(5)
test_idx = sorted(random.sample(user_idx,test_amount))
print(test_idx)
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
print(avg)

user_category after normalized by max...
user_category_norm shape  (1489, 17)
user cateogory norm [[0.         1.         0.         ... 0.05714286 0.         0.        ]
 [0.24390244 0.         0.02439024 ... 0.         0.         0.09756098]
 [0.04210526 0.04210526 0.05263158 ... 0.02105263 0.         0.02105263]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01408451 0.01408451 0.04225352 ... 0.02816901 0.         0.02816901]
 [0.03703704 0.22222222 0.14814815 ... 0.         0.         0.        ]]
Min number of followings  5
Max number of followings  34
[3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512, 513, 523, 524, 540, 564, 572, 592, 605, 611, 617, 626, 627, 633, 634, 639, 642, 646, 648, 679, 692, 696, 697, 704, 705, 726, 727, 7

In [12]:
def getScoreMatrix(RS):
    #取出test的資料
    print('RS shape',RS.shape)
    testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*18
    target = np.zeros((test_amount,yt_test_amount))
    #test_t 是true的
    #test_f 是false的

    for z in range(test_amount):
        user_id = test_idx[z]
        #positive target YouTuber list
        youtube_t = test_t[z] 
        #not target YouTuber list
        youtube_f = test_f[z]

        #前兩個放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = user_following[user_id][youtube_t[i]]
        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
            target[z][i+len(youtube_t)] = user_following[user_id][youtube_f[i]]
    sumtarget = 0
    for i in range(len(target)):
        #print(np.sum(target[i]))
        sumtarget += np.sum(target[i])
    print('num of positive data in testing:',sumtarget)
    print('total testing data:',test_amount*yt_test_amount)
    return target, testRS,sumtarget

def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [13]:
def getTOP1(target,testRS,sumtarget):
    print('top1')
    correct = 0
    for i in range(len(testRS)):
        top_0 = topN(list(testRS[i]),1) #取一個
        
        #print(top_0)
        if top_0[0] < int(np.sum(target[i])):
            correct += 1
    top1_prec = correct/len(testRS)
    top1_recall = correct/(sumtarget)
    print('prec ',top1_prec,'recall ',top1_recall)
    #f1 score
    print('F1_score:',F1_score(top1_prec,top1_recall))
    return top1_prec,top1_recall,F1_score(top1_prec,top1_recall)
def getTOP3(target,testRS,sumtarget):
    print('top3')
    correct = 0
    for i in range(len(testRS)):
        top_3 = topN(list(testRS[i]),3) #取一個
        
        #print(top_3)
        for j in range(len(top_3)):
            if top_3[j] < int(np.sum(target[i])):
                correct += 1
    top3_prec = correct/(len(testRS)*3)
    top3_recall = correct/(sumtarget)
    print('prec ',top3_prec,'recall ',top3_recall)
    #f1 score
    print('F1_score:',F1_score(top3_prec,top3_recall))
    return top3_prec,top3_recall,F1_score(top3_prec,top3_recall)
def getTOP5(target,testRS,sumtarget):
    print('top5')
    correct = 0
    for i in range(len(testRS)):
        top_5 = topN(list(testRS[i]),5) #取一個
       
        #print(top_5)
        for j in range(len(top_5)):
            if top_5[j] < int(np.sum(target[i])):
                correct += 1
    top5_prec = correct/(len(testRS)*5)
    top5_recall = correct/(sumtarget)
    print('prec ',top5_prec,'recall ',top5_recall)
    #f1 score
    print('F1_score:',F1_score(top5_prec,top5_recall))
    return top5_prec,top5_recall,F1_score(top5_prec,top5_recall)

In [14]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
# pre_list
"""
test_amount = 150
yt_test_amount = 18
"""
def NDCG(target,testRS): #target是真正的喜好
    all_sort = []
    num_ndcg = 10
    pre_matrix = np.zeros(shape=(test_amount,yt_test_amount)) #(150,18)
    for i in range(test_amount): #user amount = 150
        top_n = topN(list(testRS[i]),num_ndcg) #取10個
        #print(top_n)
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))
        #print('all_sort',topN(list(testRS[i]),len(testRS[i])))
        for j in range(len(top_n)):
            pre_matrix[i][top_n[j]] = 1

    #Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
    def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
        idcg=0
        print('ideal',ideal_list)
        for i in range(len(ideal_list)):
            #print((2**true_list[i]-1),math.log2(i+2))
            idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
        #print('idcg',idcg)
        return idcg
    def DCG(prec_list): #找出前n名的[1,1,1,0,...]
        dcg=0
        print('prec',prec_list)
        for i in range(len(prec_list)):
            dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
        #print('dcg',dcg)
        return dcg
    total_ndcg = 0
    
    for m in range(test_amount): # the number of testing users
        idcg = IDCG(target[m][:num_ndcg])
        pre_list = []
        least_pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        for s in all_sort[m][num_ndcg:]:
            #print(s)
            #print(target[m][s])
            least_pre_list.append(target[m][s]) #把prec_list 的 score加進去
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        print(ndcg)
        total_ndcg += ndcg
    avg_ndcg = total_ndcg/test_amount
    print('NDCG:',avg_ndcg)
    return pre_matrix,avg_ndcg

# MAP
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
def MAP(target,testRS):
    print('target:',target)
    print('testRS:',testRS)
    total_prec = 0
    for u in range(test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec+=average_precision_score(y_true, y_scores)
    Map_value = total_prec/test_amount
    print('MAP',Map_value)
    return Map_value

In [15]:
def testing(U, Y, A, Au, Ay, Aa):
    
    test_amount = 150
    yt_test_amount = 18
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(np.maximum(alpha, 0),0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)
        
    return RS

In [38]:
#With Embedding
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/npy/mask_feature/result300/'
all_files = os.listdir(path)
all_files =[val for val in all_files if 'No_feature' in val if '.npz' in val]
all_files

['No_feature.npz',
 'No_feature0.0001.npz',
 'No_feature0.001.npz',
 'No_feature0.01.npz',
 'No_feature0.npz',
 'No_feature1.000001.npz',
 'No_feature1.0001.npz',
 'No_feature1.001.npz',
 'No_feature1.01.npz',
 'No_feature1.npz',
 'No_feature11.npz',
 'No_feature2.npz',
 'No_feature3.npz']

In [41]:
NDCG_List = []
MAP_List = []
DIM_List = []
TOP1_Prec = []
TOP3_Prec = []
TOP5_Prec = []
TOP1_F1 = []
TOP3_F1 = []
TOP5_F1 = []
TOP1_Recall = []
TOP3_Recall = []
TOP5_Recall = []
    
for file in all_files:
    print(file)
    par_data = np.load(path+file)
    U = par_data['U']
    Y = par_data['Y']
    A = par_data['A']
    Wu = par_data['Wu']
    Wy = par_data['Wy']
    Wa = par_data['Wa']


    RS = testing(U, Y, A, Wu, Wy, Wa)
    target,testRS,sumtarget = getScoreMatrix(RS)
    prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
    prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
    prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
    pre_matrix,avg_ndcg = NDCG(target,testRS)
    Map_value = MAP(target,testRS)
    NDCG_List.append(avg_ndcg)
    MAP_List.append(Map_value)
    DIM_List.append(file.split('.npz')[0])
    TOP1_Prec.append(prec_1)
    TOP1_Recall.append(recall_1)
    TOP1_F1.append(f1_1)
    TOP3_Prec.append(prec_3)
    TOP3_Recall.append(recall_3)
    TOP3_F1.append(f1_3)
    TOP5_Prec.append(prec_5)
    TOP5_Recall.append(recall_5)
    TOP5_F1.append(f1_5)
    print('--------------------------------------------------------------------------------------------')
result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
df = pd.DataFrame(result_dict)
df.to_csv(path+file.split('.')[0]+'.csv',index=False)

No_feature.npz
RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.7133333333333334 recall  0.13048780487804879
F1_score: 0.22061855670103095
top3
prec  0.7044444444444444 recall  0.38658536585365855
F1_score: 0.49921259842519683
top5
prec  0.6573333333333333 recall  0.6012195121951219
F1_score: 0.6280254777070063
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.9709286432396583
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
0.7009124514018641
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
0.9876712785649564
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.8738100898801323
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
0.8400853550530741
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6066666666666667 recall  0.11097560975609756
F1_score: 0.18762886597938144
top3
prec  0.5266666666666666 recall  0.2890243902439024
F1_score: 0.37322834645669284
top5
prec  0.4786666666666667 recall  0.4378048780487805
F1_score: 0.4573248407643312
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.9267582364714126
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0]
0.5271763474466683
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
0.77948825478451
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.7756199604526248
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.5166400850076802
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6 recall  0.10975609756097561
F1_score: 0.18556701030927836
top3
prec  0.5177777777777778 recall  0.2841463414634146
F1_score: 0.3669291338582677
top5
prec  0.48933333333333334 recall  0.4475609756097561
F1_score: 0.467515923566879
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1.0
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
0.5058175983477707
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
0.40086221149511014
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.8013122149737618
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
0.6023254501543346
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
0.843706044175124
ideal [1.

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.6 recall  0.10975609756097561
F1_score: 0.18556701030927836
top3
prec  0.5444444444444444 recall  0.29878048780487804
F1_score: 0.3858267716535433
top5
prec  0.49733333333333335 recall  0.4548780487804878
F1_score: 0.4751592356687898
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.9828920819566879
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0]
0.5361301939825751
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
0.7285528898720137
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
0.9456352704673577
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
0.3861817348048538
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.832002041

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.7733333333333333 recall  0.14146341463414633
F1_score: 0.23917525773195872
top3
prec  0.6955555555555556 recall  0.38170731707317074
F1_score: 0.49291338582677163
top5
prec  0.628 recall  0.574390243902439
F1_score: 0.6
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.9709286432396583
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
0.7346671354126629
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.7526136409516659
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.9330795243082165
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
0.6924048374257171
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
0.971476156593264
ideal [

...

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.5133333333333333 recall  0.09390243902439024
F1_score: 0.15876288659793814
top3
prec  0.5022222222222222 recall  0.275609756097561
F1_score: 0.35590551181102364
top5
prec  0.5 recall  0.4573170731707317
F1_score: 0.4777070063694268
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
0.8036070421010475
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
0.7365447429771821
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
0.6419711018425814
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0]
0.6419273807555542
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
0.5590851016027567
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
0.77618726054

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.74 recall  0.1353658536585366
F1_score: 0.22886597938144332
top3
prec  0.6888888888888889 recall  0.3780487804878049
F1_score: 0.48818897637795267
top5
prec  0.6333333333333333 recall  0.5792682926829268
F1_score: 0.6050955414012739
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.9709286432396583
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
0.9083053473410256
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.983303248698947
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
0.8990671370233444
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.8090791338210652
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]
0.87856937699

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.7666666666666667 recall  0.1402439024390244
F1_score: 0.2371134020618557
top3
prec  0.6888888888888889 recall  0.3780487804878049
F1_score: 0.48818897637795267
top5
prec  0.6493333333333333 recall  0.5939024390243902
F1_score: 0.6203821656050955
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.9828920819566879
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
0.7045143510250984
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.983303248698947
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.8886733622104969
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
0.8363700718155497
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]


RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.8 recall  0.14634146341463414
F1_score: 0.24742268041237112
top3
prec  0.72 recall  0.3951219512195122
F1_score: 0.5102362204724409
top5
prec  0.636 recall  0.5817073170731707
F1_score: 0.6076433121019108
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
0.9709286432396583
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
0.844702407490702
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
0.9930783166417602
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.8886733622104969
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
0.7967504123860216
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.9838048780283076
ideal [1. 1. 1. 1. 1. 

RS shape (150, 88)
num of positive data in testing: 820.0
total testing data: 2700
top1
prec  0.7466666666666667 recall  0.13658536585365855
F1_score: 0.2309278350515464
top3
prec  0.7066666666666667 recall  0.3878048780487805
F1_score: 0.5007874015748032
top5
prec  0.6466666666666666 recall  0.5914634146341463
F1_score: 0.6178343949044586
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.9828920819566879
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
0.9318171001351273
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
0.9930783166417602
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
0.8886733622104969
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
0.8417771098923534
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]